In [1]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)
import matplotlib.pyplot as plt

#1. Loading the data set

In [4]:
df_ratings = pd.read_csv ('https://raw.githubusercontent.com/NatalieB-github/CE888/master/lab4-recommender/jester-data-1.csv', header=None)
df_ratings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

##1.1 Which value you should replace with NaNs?

In [45]:
df_ratings=df_ratings.replace(99.00, np.NaN)
df_ratings.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.42,NaN,NaN,NaN,-7.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,NaN,8.25,NaN,NaN,7.48,7.28,7.28,8.93,NaN,6.17,7.28,NaN,NaN,8.98,7.33,NaN,6.17,9.08,7.33,...,6.46,7.28,NaN,NaN,7.04,7.28,NaN,7.28,8.25,NaN,NaN,NaN,NaN,NaN,NaN,8.93,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.03,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,NaN,NaN,NaN,NaN,2.91,3.93,6.75,6.60,NaN,6.65,-6.12,NaN,7.57,6.21,6.65,NaN,-8.30,7.18,2.82,...,0.00,-3.69,NaN,NaN,7.82,0.24,NaN,7.28,-2.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.63,NaN,NaN,-2.33,NaN,NaN,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,NaN,NaN,NaN,NaN,NaN,NaN,4.13,NaN,NaN,NaN,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


##1.2 Removing a column

In [5]:
df_ratings.drop(columns=[0],axis=1,inplace=True)

##1.3 The best- and the worst-rated jokes

In [9]:
mean_ratings = df_ratings.mean(axis=0)
mean_ratings

1      0.904573
2      0.208460
3      0.316574
4     -1.449114
5      0.403556
         ...   
96     1.528003
97     1.673750
98     0.767692
99    -0.031244
100    1.354711
Length: 100, dtype: float64

In [17]:
pd.DataFrame(mean_ratings).nsmallest(10,0)


,0
58,-3.833880
16,-3.104818
44,-2.111891
57,-1.990664
13,-1.762882
15,-1.706201
24,-1.687420
74,-1.553548
4,-1.449114
33,-1.368648


In [18]:
pd.DataFrame(mean_ratings).nlargest(10,0)


,0
50,3.665085
89,3.574720
36,3.312500
27,3.189821
32,3.155988
35,3.006351
62,2.976021
29,2.972455
53,2.939463
49,2.771298


##1.4 Modify and use the helper function

In [20]:
arr = df_ratings.values
print(arr.shape)

(24983, 100)


In [21]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

2498300 (2498300,)


In [23]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [28]:
new_arr, idx = replace(arr, 0.1)
new_arr

array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan, 99.  ,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ..., 99.  ,   nan,   nan]])

In [25]:
arr[idx[0][0], idx[1][0]]

4.03

In [26]:
new_arr[idx[0][0], idx[1][0]]

99.0

##1.5 Use latent factor modeling

In [29]:
n_latent_factors = 2

latent_user_preferences = np.random.random((arr.shape[0], n_latent_factors))
latent_item_features = np.random.random((arr.shape[1], n_latent_factors))

In [30]:
latent_user_preferences

array([[0.409, 0.001],
       [0.695, 0.946],
       [0.611, 0.391],
       ...,
       [0.447, 0.624],
       [0.949, 0.342],
       [0.821, 0.402]])

In [31]:
latent_item_features

array([[0.167, 0.375],
       [0.026, 0.122],
       [0.242, 0.796],
       [0.273, 0.81 ],
       [0.916, 0.207],
       [0.242, 0.216],
       [0.77 , 0.123],
       [0.761, 0.484],
       [0.785, 0.263],
       [0.381, 0.276],
       [0.444, 0.437],
       [0.655, 0.428],
       [0.236, 0.303],
       [0.12 , 0.386],
       [0.822, 0.047],
       [0.996, 0.358],
       [0.295, 0.37 ],
       [0.849, 0.444],
       [0.475, 0.529],
       [0.55 , 0.906],
       [0.743, 0.007],
       [0.361, 0.37 ],
       [0.438, 0.587],
       [0.691, 0.079],
       [0.513, 0.601],
       [0.387, 0.604],
       [0.994, 0.419],
       [0.489, 0.13 ],
       [0.507, 0.009],
       [0.469, 0.491],
       [0.226, 0.591],
       [0.287, 0.66 ],
       [0.07 , 0.168],
       [0.502, 0.595],
       [0.449, 0.939],
       [0.377, 0.078],
       [0.61 , 0.616],
       [0.742, 0.78 ],
       [0.244, 0.973],
       [0.685, 0.148],
       [0.996, 0.09 ],
       [0.747, 0.916],
       [0.754, 0.668],
       [0.5

In [36]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    
def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = arr[user_id, item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [ ]:
num_iter = 10
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 10), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

In [38]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[0.558, 0.307, 0.286, ..., 0.552, 0.177, 0.852],
       [1.369, 0.704, 0.77 , ..., 1.585, 0.528, 2.204],
       [1.449, 0.765, 0.787, ..., 1.583, 0.52 , 2.286],
       ...,
       [0.924, 0.474, 0.521, ..., 1.074, 0.358, 1.49 ],
       [1.253, 0.678, 0.657, ..., 1.291, 0.418, 1.938],
       [1.432, 0.764, 0.766, ..., 1.527, 0.499, 2.241]])

In [40]:
values = [zip(arr[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df_ratings.columns

In [41]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.5578843830351572)","(8.79, 0.3068633863865638)","(-9.66, 0.2859759901261543)","(-8.16, -0.5141197959650149)","(-7.52, 0.3939346719514587)","(-8.5, 0.9572755322052946)","(-9.85, -0.023204768107098445)","(4.17, -0.16381915504370503)","(-8.98, -0.06504520137061853)","(-4.76, 0.7418991698538746)","(-8.5, 1.0667307164181552)","(-6.75, 0.9539643556138931)","(-7.18, -0.7174110140929102)","(8.45, 0.9173818211192633)","(-7.18, -0.7111164595385665)","(-7.52, -1.3181032246189943)","(-7.43, -0.41674688191001535)","(-9.81, -0.176171756901335)","(-9.85, 0.23125244264891154)","(-9.85, -0.2685548945339382)","(-9.37, 1.2835239716717002)","(1.5, 0.6100137754519391)","(-4.37, 0.23155722460817865)","(-9.81, -0.5772232070473206)","(-8.5, 0.40218567953894746)","(1.12, 0.8171313641685363)","(7.82, 1.7410545576816296)","(2.86, 0.9781970902903281)","(9.13, 1.6337570471533829)","(-7.43, -0.030671185695910156)","(2.14, 1.23088472000086)","(-4.08, 1.6980174259449932)","(-9.08, -0.41768628773528216)","(7.82, 0.6275693289671977)","(5.05, 1.6317259371902604)","(4.95, 1.7488834838865526)","(-9.17, -0.4779080702257372)","(-8.4, 0.8052719510704142)","(-8.4, 0.7494746698116882)","(-8.4, 0.7452289695408302)",...,"(8.59, 1.406150766192447)","(3.59, 1.6492763710121119)","(-6.84, 0.35429286238828306)","(-9.03, -0.08280987023671998)","(2.82, 1.3461053089646262)","(-1.36, 1.4139047025237161)","(-9.08, -0.23994101589579603)","(8.3, 1.4859962865614234)","(5.68, 1.4188949158401507)","(-4.81, 0.38135243947317576)","(nan, -0.046529441963237606)","(nan, 1.5532406920524369)","(nan, 0.7191707242808858)","(nan, -0.43754468758526355)","(nan, 0.021886661100615462)","(nan, 1.4189130975612163)","(nan, 0.5302473713202189)","(-9.42, 1.0553723472489542)","(nan, 0.2600722595256004)","(nan, 0.795759405566768)","(nan, 1.1127750905788292)","(-7.72, 0.6156650197991773)","(nan, 1.1623482825911153)","(nan, 0.5469631443646866)","(nan, 0.6651078984620312)","(nan, 0.251699777526535)","(nan, 1.0730548828402615)","(nan, 1.160769213058012)","(nan, 1.6961775921283828)","(nan, 0.4400205010530955)","(2.82, 1.1446419737543352)","(nan, 0.7519892757469729)","(nan, 1.3814853556315911)","(nan, 0.7137001975264722)","(nan, 0.6213734535846558)","(nan, 0.8278149946518891)","(-5.63, 1.0331783913093082)","(nan, 0.5524622696902385)","(nan, 0.17674098631118323)","(nan, 0.8523500170252103)"
1,"(4.08, 1.3693793826924137)","(-0.29, 0.7037315654739482)","(6.36, 0.7698993905506148)","(4.37, -1.2215184048310725)","(-2.38, 0.8214389291132316)","(-9.66, 2.4437252238194542)","(-0.73, -0.20427538938322368)","(-5.34, -0.5580922842086836)","(8.88, -0.37090022682091417)","(9.22, 1.7802435112032744)","(6.75, 2.5923812761616247)","(8.64, 2.3350143051930705)","(4.42, -1.9286267838804947)","(7.43, 2.2877431556892973)","(4.56, -1.9982641564989354)","(-0.97, -3.609283585042524)","(4.66, -1.0039443023925338)","(-0.68, -0.5980249392918228)","(3.3, 0.541330341712975)","(-1.21, -0.7413726972992288)","(0.87, 3.087536033444251)","(8.64, 1.5447736814824728)","(8.35, 0.5282482641379668)","(9.17, -1.726653223880359)","(0.05, 0.9787891928938549)","(7.57, 2.1688689712094074)","(4.71, 4.2463147991826355)","(0.87, 2.2919200681358602)","(-0.39, 4.003547204520985)","(6.99, 0.0392928492741738)","(6.5, 3.2368176293503277)","(-0.92, 4.20999196051113)","(7.14, -1.1815922299391697)","(9.03, 1.6766602291775237)","(-1.8, 4.086158318277034)","(0.73, 4.322968262694096)","(7.09, -1.2853187212754196)","(3.4, 1.9379142307575774)","(-0.87, 1.921191717658563)","(7.91, 1.7359695533345154)",...,"(-6.7, 3.510220352759077)","(-3.35, 4.017325038665514)","(-9.03, 0.844932720416712)","(4.47, -0.4590785746989421)","(4.08, 3.328100204660951)","(-3.83, 3.5550641076009457)","(8.74, -0.620361914934693)","(1.12, 3.61746198305

In [43]:
df = pd.read_excel('/content/movies_latent_factors.xlsx')
df

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9806,The Incredibles (2004),0.159967,-0.051123,0.311183,1.841867,-1.952736,-0.836041,0.639252,-0.413082,-0.310868,0.448225,-0.846044,-0.158192,-0.649164,0.045580,1.017861
96,10020,Beauty and the Beast (1991),1.286288,0.004957,-0.157640,2.579640,-0.042687,-0.256582,-0.007971,0.458163,0.091394,0.323339,-0.862833,-0.508185,0.994535,0.868700,-0.487076
97,36657,X-Men (2000),0.811901,-0.893280,0.668953,-0.885074,-1.070468,-0.973154,-0.037449,-0.925609,-0.122112,-0.328822,-0.163103,-0.307299,0.642105,-1.355542,0.067790
98,36658,X2: X-Men United (2003),1.161006,-0.715613,0.712633,-0.688808,-0.717785,-1.007047,-0.267268,-0.802322,-0.249239,-0.540419,-0.291422,-0.240216,0.290714,-1.217250,-0.288130
